# 02 - Transfer Preview Features Unit

**Purpose**: Detect activated preview features by comparing tenant settings with feature releases

**Inputs**:
- Delta Table: `tenant_settings` (from FUAM)
- Delta Table: `feature_releases` (from 01_Transfer_Feature_Releases_Unit)

**Outputs**:
- Delta Table: `preview_features_active`

**Frequency**: Daily (recommended)

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
from datetime import datetime
from difflib import SequenceMatcher

## Step 1: Load Feature Releases (Preview only)

In [ ]:
print("🔄 Loading preview feature releases...")

df_features = spark.read.format("delta").load("Tables/feature_releases")

# Filter to preview features only
df_preview_features = df_features.filter(F.col("is_preview") == True)

preview_count = df_preview_features.count()
print(f"✅ Loaded {preview_count} preview features")

# Show sample
df_preview_features.select("feature_id", "feature_name", "workload", "release_date").show(5, truncate=False)

## Step 2: Load Tenant Settings

In [ ]:
print("\n🔄 Loading tenant settings...")

df_tenant_settings = spark.read.format("delta").load("Tables/tenant_settings")

settings_count = df_tenant_settings.count()
print(f"✅ Loaded {settings_count} tenant settings")

# Show sample
df_tenant_settings.select("settingName", "enabled", "delegateToTenant").show(5, truncate=False)

## Step 3: Map Settings to Features

This creates a mapping between tenant setting names and preview feature names using fuzzy matching

In [ ]:
def similarity_score(a, b):
    """Calculate similarity between two strings (0-1)"""
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

def map_settings_to_features(df_settings, df_features):
    """
    Map tenant settings to preview features based on name similarity
    Returns DataFrame with matches
    """
    
    # Collect feature names for matching
    features_list = df_features.select("feature_id", "feature_name", "workload").collect()
    settings_list = df_settings.select("settingName", "enabled", "delegateToTenant").collect()
    
    matches = []
    
    for setting in settings_list:
        setting_name = setting["settingName"]
        
        # Skip if setting is not enabled
        if not setting["enabled"]:
            continue
        
        # Find best matching feature
        best_match = None
        best_score = 0.0
        
        for feature in features_list:
            feature_name = feature["feature_name"]
            
            # Calculate similarity
            score = similarity_score(setting_name, feature_name)
            
            # Also check for keyword matches
            setting_words = set(setting_name.lower().split())
            feature_words = set(feature_name.lower().split())
            common_words = setting_words & feature_words
            
            if common_words:
                score += len(common_words) * 0.1  # Boost score for common words
            
            if score > best_score and score > 0.3:  # Threshold 0.3
                best_score = score
                best_match = feature
        
        if best_match:
            matches.append({
                "setting_name": setting_name,
                "feature_id": best_match["feature_id"],
                "feature_name": best_match["feature_name"],
                "workload": best_match["workload"],
                "similarity_score": best_score,
                "is_enabled": setting["enabled"],
                "delegate_to_tenant": setting["delegateToTenant"],
                "detected_date": datetime.now()
            })
    
    return matches

In [ ]:
print("🔄 Mapping settings to preview features...")

matches = map_settings_to_features(df_tenant_settings, df_preview_features)

print(f"✅ Found {len(matches)} potential preview features activated")

# Preview matches
if matches:
    print("\n📋 Sample matches:")
    for match in matches[:5]:
        print(f"  - {match['feature_name']}")
        print(f"    → Setting: {match['setting_name']}")
        print(f"    → Similarity: {match['similarity_score']:.2f}")
        print()

## Step 4: Create DataFrame with Activated Previews

In [ ]:
schema = StructType([
    StructField("setting_name", StringType(), False),
    StructField("feature_id", StringType(), False),
    StructField("feature_name", StringType(), False),
    StructField("workload", StringType(), True),
    StructField("similarity_score", DoubleType(), False),
    StructField("is_enabled", BooleanType(), False),
    StructField("delegate_to_tenant", BooleanType(), True),
    StructField("detected_date", TimestampType(), False)
])

df_active_previews = spark.createDataFrame(matches, schema=schema)

print(f"✅ Created DataFrame with {df_active_previews.count()} activated preview features")
df_active_previews.show(5, truncate=False)

## Step 5: Enrich with Feature Details

In [ ]:
print("🔄 Enriching with feature release details...")

# Join with feature_releases to get release_date, status, etc.
df_enriched = df_active_previews.join(
    df_preview_features.select(
        "feature_id",
        "release_date",
        "status",
        "source_url"
    ),
    on="feature_id",
    how="left"
)

# Calculate days since release
df_enriched = df_enriched.withColumn(
    "days_since_release",
    F.datediff(F.current_date(), F.col("release_date"))
)

print("✅ Enrichment completed")
df_enriched.show(5, truncate=False)

## Step 6: Write to Delta Lake

In [ ]:
table_name = "preview_features_active"
table_path = f"Tables/{table_name}"

print(f"🔄 Writing to Delta table: {table_name}")

try:
    from delta.tables import DeltaTable
    
    # Check if table exists
    if DeltaTable.isDeltaTable(spark, table_path):
        print("  → Table exists, performing MERGE (upsert)...")
        
        delta_table = DeltaTable.forPath(spark, table_path)
        
        # Merge: Update existing, insert new
        delta_table.alias("target").merge(
            df_enriched.alias("source"),
            "target.feature_id = source.feature_id AND target.setting_name = source.setting_name"
        ).whenMatchedUpdate(
            set={
                "is_enabled": "source.is_enabled",
                "detected_date": "source.detected_date",
                "days_since_release": "source.days_since_release"
            }
        ).whenNotMatchedInsertAll(
        ).execute()
        
        print("  ✅ MERGE completed")
    else:
        print("  → Table doesn't exist, creating new table...")
        df_enriched.write.format("delta").mode("overwrite").save(table_path)
        print("  ✅ Table created")
    
    # Show final count
    final_count = spark.read.format("delta").load(table_path).count()
    print(f"\n✅ Total activated preview features in {table_name}: {final_count}")
    
except Exception as e:
    print(f"❌ Error writing to Delta: {e}")
    raise

## Step 7: Summary Statistics

In [ ]:
print("\n📊 Activated Preview Features Statistics:")
print("=" * 60)

df_summary = spark.read.format("delta").load(table_path)

# By workload
print("\n🔸 By Workload:")
df_summary.groupBy("workload").count().orderBy(F.desc("count")).show(truncate=False)

# By similarity score range
print("\n🔸 By Confidence Level (Similarity Score):")
df_summary.withColumn(
    "confidence",
    F.when(F.col("similarity_score") >= 0.7, "High (≥0.7)")
     .when(F.col("similarity_score") >= 0.5, "Medium (0.5-0.7)")
     .otherwise("Low (<0.5)")
).groupBy("confidence").count().orderBy(F.desc("count")).show(truncate=False)

# Recently released (last 30 days) but already activated
recent_activated = df_summary.filter(F.col("days_since_release") <= 30).count()
print(f"\n🔸 Features activated within 30 days of release: {recent_activated}")

# Features that have been in preview for long time
long_preview = df_summary.filter(F.col("days_since_release") > 180).count()
print(f"🔸 Features in preview for >180 days: {long_preview}")

print("\n" + "=" * 60)
print("✅ Transfer Preview Features Unit - COMPLETED")